In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('data_processing').getOrCreate()

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [4]:
df=spark.read.csv("/FileStore/tables/customers1.csv",header=True,inferSchema=True)

In [5]:
df.show(5)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
 Customer_subtype|Number_of_houses|Avg_size_household| Avg_age| Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
Lower class large...| 1| 3|30-40 years|Family with grown...| 44905| 0|
Mixed small town ...| 1| 2|30-40 years|Family with grown...| 37575| 0|
Mixed small town ...| 1| 2|30-40 years|Family with grown...| 27915| 0|
Modern, complete ...| 1| 3|40-50 years| Average Family| 19504| 0|
 Large family farms| 1| 4|30-40 years| Farmers| 34943| 0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
only showing top 5 rows

In [6]:
region_data = spark.createDataFrame([('Family with grownups','PN'), \
  ('Driven Growers','GJ'), \
  ('Conservative families','DD'), \
  ('Cruising Seniors','DL'), \
  ('Average Family ','MN'), \
  ('Living well','KA'), \
  ('Successful hedonists','JH'), \
  ('Retired and Religious','AX'), \
  ('Career Loners','HY'),('Farmers','JH')], \
  schema=StructType().add("Customer_main_type","string").add("Region Code","string"))


In [7]:
region_data.show(3,truncate=False)

+---------------------+-----------+
Customer_main_type |Region Code|
+---------------------+-----------+
Family with grownups |PN |
Driven Growers |GJ |
Conservative families|DD |
+---------------------+-----------+
only showing top 3 rows

In [8]:
new_df = df.join(region_data,on='Customer_main_type') #Joining by main type column

In [9]:
new_df.groupBy("Region Code").count().show()


+-----------+-----+
Region Code|count|
+-----------+-----+
 JH| 287|
 HY| 15|
 DD| 236|
 DL| 60|
 GJ| 172|
 KA| 178|
 AX| 202|
+-----------+-----+

In [10]:
new_df.groupBy('Customer_main_type').pivot('Region Code').sum('Avg_Salary').fillna(0).show()

+--------------------+-------+-------+-------+-------+------+----------+-------+
 Customer_main_type| AX| DD| DL| GJ| HY| JH| KA|
+--------------------+-------+-------+-------+-------+------+----------+-------+
 Farmers| 0| 0| 0| 0| 0| 2809468| 0|
 Career Loners| 0| 0| 0| 0|484089| 0| 0|
Retired and Relig...|5522439| 0| 0| 0| 0| 0| 0|
Successful hedonists| 0| 0| 0| 0| 0|3158111161| 0|
 Living well| 0| 0| 0| 0| 0| 0|5552540|
 Cruising Seniors| 0| 0|1732220| 0| 0| 0| 0|
Conservative fami...| 0|6963043| 0| 0| 0| 0| 0|
 Driven Growers| 0| 0| 0|5292275| 0| 0| 0|
+--------------------+-------+-------+-------+-------+------+----------+-------+

In [11]:
new_df.groupBy('Customer_main_type').pivot('AVG_age').sum('Avg_Salary').fillna(0).show()

+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
 Customer_main_type|20-30 years|30-40 years|40-50 years|50-60 years|60-70 years|70-80 years|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
 Farmers| 0| 462027| 2031235| 316206| 0| 0|
 Career Loners| 143998| 176639| 25701| 105193| 32558| 0|
Retired and Relig...| 126350| 336631| 2975266| 1687711| 335357| 61124|
Successful hedonists| 42261| 171278764| 1223362814| 1563071675| 200340129| 15518|
 Living well| 460528| 2965303| 1795405| 331304| 0| 0|
 Cruising Seniors| 0| 43302| 303601| 529354| 716425| 139538|
Conservative fami...| 69390| 2381485| 3449782| 915954| 146432| 0|
 Driven Growers| 0| 1376260| 3407807| 424272| 83936| 0|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+

In [12]:
def age_category(age):
  if age == "20-30 years":
    return "Young"
  elif age== "30-40 years":
    return "Mid Aged"
  elif ((age== "40-50 years") or (age== "50-60 years")) :
    return "Old"
  else:
    return "Very Old"
  
age_udf=udf(age_category,StringType())
new_df=new_df.withColumn('age_category',age_udf(new_df['Avg_age']))

In [13]:
new_df.select('Avg_age','age_category').show()

+-----------+------------+
 Avg_age|age_category|
+-----------+------------+
40-50 years| Old|
50-60 years| Old|
40-50 years| Old|
30-40 years| Mid Aged|
30-40 years| Mid Aged|
40-50 years| Old|
40-50 years| Old|
30-40 years| Mid Aged|
40-50 years| Old|
40-50 years| Old|
40-50 years| Old|
30-40 years| Mid Aged|
40-50 years| Old|
40-50 years| Old|
40-50 years| Old|
30-40 years| Mid Aged|
30-40 years| Mid Aged|
40-50 years| Old|
40-50 years| Old|
50-60 years| Old|
+-----------+------------+
only showing top 20 rows

In [14]:
new_df.groupBy('Customer_main_type').pivot('label').sum('Avg_Salary').fillna(0).show()

+--------------------+----------+---------+
 Customer_main_type| 0| 1|
+--------------------+----------+---------+
 Farmers| 2734832| 74636|
 Career Loners| 484089| 0|
Retired and Relig...| 5328410| 194029|
Successful hedonists|2720381462|437729699|
 Living well| 5453384| 99156|
 Cruising Seniors| 1675841| 56379|
Conservative fami...| 6595027| 368016|
 Driven Growers| 4492465| 799810|
+--------------------+----------+---------+

In [15]:
# This example shows a 'Collect list'. A collect list is a list of all values. See the second column in the result.
new_df.groupby("Customer_subtype").agg(F.collect_list("Number_of_houses")).show()

+--------------------+------------------------------+
 Customer_subtype|collect_list(Number_of_houses)|
+--------------------+------------------------------+
Religious elderly...| [1, 1, 1, 1, 1, 1...|
Large religious f...| [1, 1, 1, 1, 1, 2...|
Young urban have-...| [1, 1, 2, 1]|
Young seniors in ...| [1, 3, 1, 1, 1, 1...|
Fresh masters in ...| [1, 1]|
High Income, expe...| [1, 1, 1, 1, 1, 1...|
 Residential elderly| [1, 2, 3, 1, 1, 3]|
Senior cosmopolitans| [3]|
 Mixed rurals| [1, 1, 1, 1, 1, 1...|
Career and childcare| [1, 1, 1, 1, 1, 2...|
Low income catholics| [1, 1, 1, 1, 1, 1...|
Mixed apartment d...| [1, 1, 1, 1, 1, 2...|
Seniors in apartm...| [2, 2, 1, 1, 1, 2...|
Middle class fami...| [1, 1, 1, 1, 1, 1...|
Traditional families| [1, 1, 1, 1, 1, 1...|
Porchless seniors...| [1, 1, 1, 1, 1, 1...|
 Young, low educated| [1, 1, 2, 1, 1, 1...|
 Own home elderly| [1, 1, 1, 1, 1, 1...|
 Young and rising| [1, 1, 1, 1, 1, 1...|
 High status seniors| [1, 2, 1, 1, 1, 1...|
+--------------------+------------------------------+
only showing top 20 rows

In [16]:
# This example shows a 'Collect set'. A collect set is a set with unique values in a set. See the second column in the result.
new_df.groupby("Customer_subtype").agg(F.collect_set("Number_of_houses")).show()

+--------------------+-----------------------------+
 Customer_subtype|collect_set(Number_of_houses)|
+--------------------+-----------------------------+
Religious elderly...| [1, 2]|
Large religious f...| [1, 2]|
Young urban have-...| [1, 2]|
Young seniors in ...| [1, 2, 3]|
Fresh masters in ...| [1]|
High Income, expe...| [1]|
 Residential elderly| [1, 2, 3]|
Senior cosmopolitans| [3]|
 Mixed rurals| [1]|
Career and childcare| [1, 2]|
Low income catholics| [1]|
Mixed apartment d...| [1, 2, 3]|
Seniors in apartm...| [1, 2]|
Middle class fami...| [1, 2]|
Traditional families| [1, 2]|
Porchless seniors...| [1]|
 Young, low educated| [1, 2]|
 Own home elderly| [1, 2, 3]|
 Young and rising| [1, 5, 2]|
 High status seniors| [1, 2]|
+--------------------+-----------------------------+
only showing top 20 rows

In [17]:
new_df.groupBy('Region Code').agg(F.avg('Avg_Salary').alias('mean_salary')).orderBy('mean_salary',ascending=False).show(50,False)

+-----------+--------------------+
Region Code|mean_salary |
+-----------+--------------------+
JH |1.1013660728222996E7|
HY |32272.6 |
KA |31194.044943820223 |
GJ |30769.04069767442 |
DD |29504.419491525423 |
DL |28870.333333333332 |
AX |27338.80693069307 |
+-----------+--------------------+

In [18]:
new_df.groupBy('Customer_subtype').agg(F.avg('Avg_Salary').alias('mean_salary')).orderBy('mean_salary',ascending=False).show(50,False)

+--------------------------------+--------------------+
Customer_subtype |mean_salary |
+--------------------------------+--------------------+
High status seniors |2.507677857894737E7 |
High Income, expensive child |2.3839817807692308E7|
Affluent senior apartments |653638.8235294118 |
Senior cosmopolitans |49903.0 |
Students in apartments |35532.142857142855 |
Large family farms |33135.61538461538 |
Young, low educated |33072.21428571428 |
Suburban youth |32558.0 |
Middle class families |31579.385245901638 |
Etnically diverse |31572.0 |
Young and rising |30795.897435897437 |
Mixed seniors |30759.267605633802 |
Very Important Provincials |30548.40625 |
Religious elderly singles |30540.59574468085 |
Career and childcare |30110.939393939392 |
Young seniors in the city |30105.136363636364 |
Seniors in apartments |30090.882352941175 |
Large religious families |29652.196261682242 |
Mixed apartment dwellers |29587.647058823528 |
Porchless seniors: no front yard|29509.827586206895 |
Traditional families |29381.84496124031 |
Mixed rurals |29073.761194029852 |
Residential elderly |28866.166666666668 |
Fresh masters in the city |27645.0 |
Dinki's (double income no kids) |26231.117647058825 |
Young urban have-nots |25751.0 |
Own home elderly |25677.666666666668 |
Single youth |24403.25 |
Low income catholics |21713.777777777777 |
+--------------------------------+--------------------+

In [19]:
new_df.sort("Avg_Salary", ascending=False).show()

+--------------------+--------------------+----------------+------------------+-----------+----------+-----+-----------+------------+
 Customer_main_type| Customer_subtype|Number_of_houses|Avg_size_household| Avg_age|Avg_Salary|label|Region Code|age_category|
+--------------------+--------------------+----------------+------------------+-----------+----------+-----+-----------+------------+
Successful hedonists| High status seniors| 1| 2|60-70 years| 48919896| 0| JH| Very Old|
Successful hedonists|High Income, expe...| 1| 2|50-60 years| 48177970| 0| JH| Old|
Successful hedonists|High Income, expe...| 1| 2|50-60 years| 48069548| 1| JH| Old|
Successful hedonists|High Income, expe...| 1| 3|40-50 years| 46911924| 0| JH| Old|
Successful hedonists| High status seniors| 1| 3|40-50 years| 46614009| 0| JH| Old|
Successful hedonists|High Income, expe...| 1| 3|30-40 years| 45952441| 0| JH| Mid Aged|
Successful hedonists|High Income, expe...| 1| 3|40-50 years| 45864609| 1| JH| Old|
Successful hedonists| High status seniors| 1| 1|50-60 years| 45592572| 0| JH| Old|
Successful hedonists| High status seniors| 1| 2|50-60 years| 45170899| 0| JH| Old|
Successful hedonists|High Income, expe...| 1| 3|50-60 years| 44843830| 0| JH| Old|
Successful hedonists| High status seniors| 1| 2|50-60 years| 43230349| 0| JH| Old|
Successful hedonists| High status seniors| 1| 5|40-50 years| 43181830| 0| JH| Old|
Successful hedonists| High status seniors| 1| 1|30-40 years| 42631926| 0| JH| Mid Aged|
Successful hedonists| High status seniors| 1| 2|50-60 years| 41919020| 0| JH| Old|
Successful hedonists| High status seniors| 1| 2|50-60 years| 41699271| 0| JH| Old|
Successful hedonists| High status seniors| 1| 2|40-50 years| 41398953| 0| JH| Old|
Successful hedonists|High Income, expe...| 1| 3|40-50 years| 41269615| 1| JH| Old|
Successful hedonists| High status seniors| 1| 2|50-60 years| 41192397| 1| JH| Old|
Successful hedonists| High status seniors| 1| 2|40-50 years| 40564335| 0| JH| Old|
Successful hedonists| High status seniors| 1| 2|50-60 years| 40453887| 0| JH| Old|
+--------------------+--------------------+----------------+------------------+-----------+----------+-----+-----------+------------+
only showing top 20 rows

In [20]:
new_df.groupBy('Customer_main_type').agg(F.max('Avg_Salary')).show()

+--------------------+---------------+
 Customer_main_type|max(Avg_Salary)|
+--------------------+---------------+
 Farmers| 49965|
 Career Loners| 49903|
Retired and Relig...| 49564|
Successful hedonists| 48919896|
 Living well| 49816|
 Cruising Seniors| 49526|
Conservative fami...| 49965|
 Driven Growers| 49932|
+--------------------+---------------+

In [21]:
# GroupBy each column except Avg_Salary.
for col in df.columns:
  if col !='Avg_Salary':
    print(f" Aggregation for {col}")
    new_df.groupBy(col).count().orderBy('count',ascending=False).show(truncate=False)

Aggregation for Customer_subtype
+--------------------------------+-----+
Customer_subtype |count|
+--------------------------------+-----+
Traditional families |129 |
Middle class families |122 |
Large religious families |107 |
Young and rising |78 |
High status seniors |76 |
Low income catholics |72 |
Mixed seniors |71 |
Mixed rurals |67 |
Young, low educated |56 |
High Income, expensive child |52 |
Religious elderly singles |47 |
Mixed apartment dwellers |34 |
Career and childcare |33 |
Very Important Provincials |32 |
Porchless seniors: no front yard|29 |
Large family farms |26 |
Young seniors in the city |22 |
Affluent senior apartments |17 |
Seniors in apartments |17 |
Dinki's (double income no kids) |17 |
+--------------------------------+-----+
only showing top 20 rows

 Aggregation for Number_of_houses
+----------------+-----+
Number_of_houses|count|
+----------------+-----+
1 |1036 |
2 |101 |
3 |11 |
5 |1 |
10 |1 |
+----------------+-----+

 Aggregation for Avg_size_household
+------------------+-----+
Avg_size_household|count|
+------------------+-----+
2 |473 |
3 |462 |
4 |113 |
1 |83 |
5 |19 |
+------------------+-----+

 Aggregation for Avg_age
+-----------+-----+
Avg_age |count|
+-----------+-----+
40-50 years|542 |
30-40 years|276 |
50-60 years|245 |
60-70 years|50 |
20-30 years|30 |
70-80 years|7 |
+-----------+-----+

 Aggregation for Customer_main_type
+---------------------+-----+
Customer_main_type |count|
+---------------------+-----+
Conservative families|236 |
Retired and Religious|202 |
Successful hedonists |194 |
Living well |178 |
Driven Growers |172 |
Farmers |93 |
Cruising Seniors |60 |
Career Loners |15 |
+---------------------+-----+

 Aggregation for label
+-----+-----+
label|count|
+-----+-----+
0 |1079 |
1 |71 |
+-----+-----+

 Aggregation for age_category
+------------+-----+
age_category|count|
+------------+-----+
Old |787 |
Mid Aged |276 |
Very Old |57 |
Young |30 |
+------------+-----+

In [22]:
new_df.groupBy('Customer_subtype').count().show()

+--------------------+-----+
 Customer_subtype|count|
+--------------------+-----+
Religious elderly...| 47|
Large religious f...| 107|
Young urban have-...| 4|
Young seniors in ...| 22|
Fresh masters in ...| 2|
High Income, expe...| 52|
 Residential elderly| 6|
Senior cosmopolitans| 1|
 Mixed rurals| 67|
Career and childcare| 33|
Low income catholics| 72|
Mixed apartment d...| 34|
Seniors in apartm...| 17|
Middle class fami...| 122|
Traditional families| 129|
Porchless seniors...| 29|
 Young, low educated| 56|
 Own home elderly| 15|
 Young and rising| 78|
 High status seniors| 76|
+--------------------+-----+
only showing top 20 rows